In [113]:
import math
import numpy as np

import torch
import gpytorch
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from gpytorch.mlls.variational_elbo import VariationalELBO

from sklearn.cross_validation import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

SEED = 199 

In [114]:
X = np.load('X.npy')
y = np.load('y.npy')

print(X.shape, y.shape,sep='\n')
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=.33, random_state=SEED)

(20468, 91)
(20468,)


In [115]:
class GPClassifier(AbstractVariationalGP):
    def __init__(self, X_train):
        variational_distribution = CholeskyVariationalDistribution(X_train.size(0))
        variational_strategy = VariationalStrategy(self, X_train, variational_distribution)
        super(GPClassifier, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return latent_pred


In [145]:
X_train,y_train = torch.FloatTensor(X_train), torch.FloatTensor(y_train)
# Initialize model and likelihood
model = GPClassifier(X_train)
likelihood = gpytorch.likelihoods.BernoulliLikelihood()

In [142]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs - the marginal log likelihood
# num_data refers to the amount of training data
mll = VariationalELBO(likelihood, model, y_train.size(0))

training_iter = 50
for i in range(training_iter):
    # Zero backpropped gradients from previous iteration
    optimizer.zero_grad()
    # Get predictive output
    output = model(X_train)
    # Calc loss and backprop gradients
    loss = -mll(output, y_train)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iter, loss.item()))
    optimizer.step()

RuntimeError: Lapack Error in potrf : the leading minor of order 5 is not positive definite at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensorLapack.cpp:675